In [1]:
# Importing Libs
import pandas as pd
import pickle
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split

In [2]:
# Loading the dataset
data=pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
# Preprocssing the data
data = data.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [4]:
# Label Encoding the Gender
le_gender=LabelEncoder() 
data['Gender']=le_gender.fit_transform(data['Gender'])

In [5]:
# OneHot Encoding the Geography
ohe_geo=OneHotEncoder()
geo_encoder=ohe_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(
    geo_encoder,
    columns=['France', 'Germany', 'Spain']
)
geo_encoded_df

,France,Germany,Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [6]:
## Combining the OHE with OG data
data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,France,Germany,Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [7]:
## Dividing the Data into Independent and Dependent Features 
X=data.drop('EstimatedSalary',axis=1)
y=data['EstimatedSalary']

In [8]:
## Split Train & Test feature
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [9]:
## Scaling the Datas
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [10]:
## Saving Encoders and Scaler into pickle file
with open('le_gender_salary.pkl','wb') as file:
    pickle.dump(le_gender,file)

with open('ohe_geo_salary.pkl','wb') as file:
    pickle.dump(ohe_geo,file)

with open('scaler_salary.pkl','wb') as file:
    pickle.dump(scaler,file)


## ANN with Regression 

In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [12]:
## Building ANN Model

model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)), ## Hidden Layer 1 Connected with Input Layer
    Dense(32,activation='relu'), ## Hidden Layer 2
    Dense(1) ## Output Regression Layer
])

## Compiling the model
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae']) 

d:\Projects\Bank Customer's Churn Prediction\.venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

## Setuping the Tensorboard
log_dir="rglogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tf_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [15]:
## Setuping the Early Stopping
es_callback=EarlyStopping(monitor='val_loss',patience=13,restore_best_weights=True)

In [16]:
## Training the Model
history=model.fit(
    X_train,
    y_train,
    validation_data=(X_test,y_test),
    epochs=100,
    callbacks=[tf_callback,es_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 100390.2266 - mae: 100390.2266 - val_loss: 98562.6328 - val_mae: 98562.6328
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 99794.2891 - mae: 99794.2891 - val_loss: 97350.3438 - val_mae: 97350.3438
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 97663.8984 - mae: 97663.8984 - val_loss: 94210.3203 - val_mae: 94210.3203
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 93404.3047 - mae: 93404.3047 - val_loss: 88833.5938 - val_mae: 88833.5938
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 87018.2969 - mae: 87018.2969 - val_loss: 81633.9297 - val_mae: 81633.9297
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 79081.2734 - mae: 79081.2734 - val_loss: 73509.0625 - val_mae: 73509.0625
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 70724.7188 - mae: 70724.7188 - val_loss: 65808.0312 - val_mae: 65808.0312
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 

In [17]:
%load_ext tensorboard

In [19]:
%tensorboard --logdir rglogs/fit/20250920-121928

Reusing TensorBoard on port 6006 (pid 8920), started 0:00:18 ago. (Use '!kill 8920' to kill it.)

In [20]:
## Evaluating the Model with test data
test_loss,test_mae=model.evaluate(X_test,y_test)
print(f'Test MAE : {test_mae}')

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 50272.8867 - mae: 50272.8867  
Test MAE : 50272.88671875


In [21]:
## Saving the model as .h5
model.save('rgmodel.h5')